## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.hidden_layer_size = 256
        self.input_size = 28 * 28
        self.output_size = 10
        self.W1 = tf.Variable(tf.random.uniform(shape=[self.input_size, self.hidden_layer_size], minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(tf.zeros(self.hidden_layer_size))
        self.W2 = tf.Variable(tf.random.uniform(shape=[self.hidden_layer_size, self.output_size], minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(tf.zeros(self.output_size))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################

        x_flattened = tf.reshape(x, [-1, self.input_size])
        hidden_layer = tf.nn.relu(tf.matmul(x_flattened, self.W1) + self.b1)
        logits = tf.matmul(hidden_layer, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        print(x.shape)
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(100):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

(60000, 28, 28)
epoch 0 : loss 2.414067 ; accuracy 0.12268333
epoch 1 : loss 2.400968 ; accuracy 0.12588333
epoch 2 : loss 2.388497 ; accuracy 0.12955
epoch 3 : loss 2.3765824 ; accuracy 0.13321666
epoch 4 : loss 2.365165 ; accuracy 0.13703333
epoch 5 : loss 2.35419 ; accuracy 0.14078334
epoch 6 : loss 2.3436127 ; accuracy 0.14433333
epoch 7 : loss 2.333394 ; accuracy 0.14875
epoch 8 : loss 2.3234994 ; accuracy 0.15341666
epoch 9 : loss 2.3138988 ; accuracy 0.15751667
epoch 10 : loss 2.304565 ; accuracy 0.16218333
epoch 11 : loss 2.2954755 ; accuracy 0.16718334
epoch 12 : loss 2.28661 ; accuracy 0.17201667
epoch 13 : loss 2.2779484 ; accuracy 0.17756666
epoch 14 : loss 2.2694757 ; accuracy 0.18341666
epoch 15 : loss 2.2611752 ; accuracy 0.18956667
epoch 16 : loss 2.2530346 ; accuracy 0.19628334
epoch 17 : loss 2.2450411 ; accuracy 0.20345
epoch 18 : loss 2.2371843 ; accuracy 0.21078333
epoch 19 : loss 2.2294528 ; accuracy 0.21826667
epoch 20 : loss 2.2218375 ; accuracy 0.22671667
epoch